In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from grammar import grammar

### RSI

RSI - wskaźnik momentum, który mierzy szybkość i zmianę ruchów cenowych.
Range - 0 do 100

$$
RSI = 100 - \frac{100}{1 + RS}
$$

Gdzie

$$
RS = \frac{\text{Average Gain}}{\text{Average Loss}}
$$

In [ ]:
def calculate_rsi(series, period):
    """RSI mowi nam o sile aktywow sugerujac czy sa overbought czy oversold"""
    delta = series.diff()

    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

In [ ]:
def get_data(ticker):
    data = yf.download(ticker, start="2020-01-01", end="2025-01-01")
    data.columns = data.columns.get_level_values(0)

    data["SMA10"] = data["Close"].rolling(10).mean()
    data["SMA50"] = data["Close"].rolling(50).mean()
    data["RSI"] = calculate_rsi(data["Close"], period=14)

    data = data.dropna()
    
    return data

In [5]:
def evaluate_condition(cond, row):
    if "logic_op" in cond:
        left_res = evaluate_condition(cond["left"], row)
        right_res = evaluate_condition(cond["right"], row)
        if cond["logic_op"] == "AND":
            return left_res and right_res
        elif cond["logic_op"] == "OR":
            return left_res or right_res

    left = row[cond["left"]] if cond["left"] in row else cond["left"]
    right = row[cond["right"]] if cond["right"] in row else cond["right"]

    if cond["op"] == ">":
        return left > right
    elif cond["op"] == "<":
        return left < right
    elif cond["op"] == ">=":
        return left >= right
    elif cond["op"] == "<=":
        return left <= right
    elif cond["op"] == "==":
        return left == right

In [6]:
def evaluate_strategy(strategy, row):
    if strategy["type"] == "action":
        return strategy["value"]
    
    condition = strategy["condition"]
    if evaluate_condition(condition, row):
        return evaluate_strategy(strategy["then"], row)
    else:
        return evaluate_strategy(strategy["else"], row)

In [ ]:
def backtest(rule, data, initial_cash):
    """
    rule - decision rules used to calculating return
    ticker - ticker on which we will be testing rule

    return: total return by using given decision rule
    """
    cash = initial_cash
    stock = 0
    for i in range(len(data)):
        price = data.iloc[i]["Close"]
        action = evaluate_strategy(rule, data.iloc[i])
        if action == "BUY" and stock == 0:
            stock = cash / price
            cash = 0
        elif action == "SELL" and stock > 0:
            cash = stock * price
            stock = 0

    total_return = cash + stock * data.iloc[-1]["Close"] - initial_cash
    return total_return

In [8]:
test_rule = {
    "type": "if",
    "condition": {
        "left": "SMA10",
        "op": ">",
        "right": "SMA50"
    },
    "then": {
        "type": "action",
        "value": "BUY"
    },
    "else": {
        "type": "action",
        "value": "SELL"
    }
}

In [ ]:
total_return = backtest(test_rule, get_data("AAPL"), initial_cash=1000)
print(f"Testowy total return: {total_return:.2f}")

[*********************100%***********************]  1 of 1 completed

Testowy total return: 1428.40


In [10]:
get_data('AAPL')

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,SMA10,SMA50
Date,,,,,,,
2020-03-13,67.227577,67.699191,61.176441,64.064157,370732000,68.346408,73.870380
2020-03-16,58.578976,62.659014,58.044481,58.516092,322423600,66.977525,73.592594
2020-03-17,61.154701,62.303495,57.657519,59.860791,324056000,66.095734,73.380413
2020-03-18,59.657635,60.463002,57.347947,57.988857,300233600,64.739669,73.126855
2020-03-19,59.200531,61.149857,58.675713,59.831765,271857200,63.575393,72.870958
...,...,...,...,...,...,...,...
2024-12-24,257.037506,257.047440,254.140589,254.339701,23234700,250.158601,234.761245
2024-12-26,257.853760,258.928914,256.470034,257.027510,27237100,251.405956,235.267494
2024-12-27,254.439224,257.535238,251.920617,256.669129,42355300,252.165518,235.746622
